<center><h2>Jane Street Market Prediction | XGB overfit | katsu1110 </h2></center><hr>

**CatBoost** has the **has_time** feature where the order of objects in the input data is kept (do not perform random permutations during the Transforming categorical features to numerical features and Choosing the tree structure stages). It sounds promising in this time-series forecasting competition.

This notebook loads feather data from [my another notebook](https://www.kaggle.com/code1110/janestreet-save-as-feather?scriptVersionId=47635784).

This notebook treats this task as a binary classification.

In [ ]:
import janestreet
import numpy as np
import pandas as pd

import os, sys
import gc
import math
import random
import pathlib
from tqdm import tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn import linear_model
import operator
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from tqdm import tqdm

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib_venn import venn2
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

# Config

In [ ]:
SEED = 2020 # The end of it is coming...
# INPUT_DIR = '../input/jane-street-market-prediction/'
INPUT_DIR = '../input/janestreet-save-as-feather/'
TRADING_THRESHOLD = 0.5 # 0 ~ 1: The smaller, the more aggressive
DATE_BEGIN = 0

# Load data

In [ ]:
os.listdir(INPUT_DIR)

In [ ]:
%%time

def load_data(input_dir=INPUT_DIR):
    train = pd.read_feather(pathlib.Path(input_dir + 'train.feather'))
    features = pd.read_feather(pathlib.Path(input_dir + 'features.feather'))
    example_test = pd.read_feather(pathlib.Path(input_dir + 'example_test.feather'))
    ss = pd.read_feather(pathlib.Path(input_dir + 'example_sample_submission.feather'))
    return train, features, example_test, ss

train, features, example_test, ss = load_data(INPUT_DIR)

In [ ]:
del features, example_test, ss
gc.collect()

# Model fitting
For now, let's use a simple XGBoost which is also used in the example in the Numerai Tournament.

In [ ]:
# remove weight = 0 for saving memory 
original_size = train.shape[0]
train = train.query('weight > 0').reset_index(drop=True)

# use data later than DATE_BEGIN
train = train.query(f'date >= {DATE_BEGIN}')

print('Train size reduced from {:,} to {:,}.'.format(original_size, train.shape[0]))

In [ ]:
# target
train['action'] = train['resp'] * train['weight']
train['action'] = 1 * (train['action'] > 0)

In [ ]:
# features to use
drops = []
feats = [f for f in train.columns.values.tolist() if f.startswith('feature') & (f not in drops)]
print('There are {:,} features.'.format(len(feats)))

In [ ]:
%%time

# same hyperparameters from https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min?scriptVersionId=48127254
params = { 
    'task_type': "CPU",
    'learning_rate': 0.08, 
    'iterations': 600,
    'colsample_bylevel': 0.2,
    'random_seed': SEED,
    'has_time': True
    }
params["loss_function"] = "Logloss"
params["eval_metric"] = "Logloss"
model = CatBoostClassifier(**params)
    
model.fit(train[feats], train['action'], verbose=100)

# Feature importance
Let's see feature importance given by the model.

In [ ]:
pd.DataFrame(model.get_feature_importance(), index=feats, columns=['importance']).sort_values(by='importance', ascending=False).style.background_gradient(cmap='viridis')

# Submit

In [ ]:
env = janestreet.make_env()
test = env.iter_test()
        
for (t, sub) in test:
    sub.action = (model.predict_proba(t[feats])[:, 1] > TRADING_THRESHOLD).astype('int')
    env.predict(sub)       

In [ ]:
# env = janestreet.make_env()
# test = env.iter_test()

# for (t, sub) in test:
#     sub.action = (model.predict(t[feats]) > TRADING_THRESHOLD - 0.5).astype('int')
#     env.predict(sub)